In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

# Load the CSV file into a DataFrame
csv_file_path = '/Users/dengzixin/Desktop/winter2024/dataAnalytics/project/test.csv'
df = pd.read_csv(csv_file_path)

chrome_options = Options()
#chrome_options.add_argument("--headless")  # Uncomment for headless mode
driver = webdriver.Chrome(options=chrome_options)
wait = WebDriverWait(driver, 10)  # Initialize WebDriverWait outside the loop

def extract_data(xpath, column_prefix, index):
    try:
        data_row = wait.until(EC.visibility_of_element_located((By.XPATH, xpath)))
        data_cells = data_row.find_elements(By.TAG_NAME, "td")[1:]  # Skip the first cell
        for i, cell in enumerate(data_cells[:5]):
            df.at[index, f'{column_prefix}Q{5-i}'] = cell.text
        missing_data_count = 5 - len(data_cells)
        for i in range(missing_data_count):  # Fill in 'DNE' for missing cells
            df.at[index, f'{column_prefix}Q{i+1}'] = 'DNE'
    except TimeoutException:
        # If a timeout occurs here, it's handled at the row level in the main loop
        pass

rows_to_drop = []  # Track rows that need to be dropped

for index, row in df.iterrows():
    try:
        driver.get(row['URL'])
        # Original data row
        extract_data("//*[@id='main-content-column']/div/div[4]/div[1]/div[2]/div/div[2]/div/div[1]/table/tbody/tr[2]", 'sales', index)

        # New data rows
        extract_data("//*[@id='main-content-column']/div/div[4]/div[1]/div[2]/div/div[2]/div/div[1]/table/tbody/tr[4]", 'netIncome', index)
        extract_data("//*[@id='main-content-column']/div/div[4]/div[2]/div[2]/div/div[2]/div/div[1]/table/tbody/tr[2]", 'totalAsset', index)
        extract_data("//*[@id='main-content-column']/div/div[4]/div[2]/div[2]/div/div[2]/div/div[1]/table/tbody/tr[4]", 'totalLiabilities', index)
        extract_data("//*[@id='main-content-column']/div/div[4]/div[3]/div[2]/div/div[2]/div/div[1]/table/tbody/tr[2]", 'opCashflow', index)
        extract_data("//*[@id='main-content-column']/div/div[4]/div[3]/div[2]/div/div[2]/div/div[1]/table/tbody/tr[4]", 'netCashflow', index)

    except TimeoutException:
        # If a timeout occurs, add the index of the row to drop
        rows_to_drop.append(index)
        continue

# Drop the rows that encountered timeouts
df.drop(index=rows_to_drop, inplace=True)

driver.quit()

# Save the updated DataFrame to a new CSV file
output_file_path = '/Users/dengzixin/Desktop/winter2024/dataAnalytics/project/test_result_1001_end.csv'
df.to_csv(output_file_path, index=False)

